In [ ]:
pip install gigachain gigachain_community gigachain-cli duckduckgo-search pyfiglet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models.gigachat import GigaChat

In [1]:
giga = GigaChat(credentials="none",
                model="GigaChat", timeout=30, verify_ssl_certs=False)
giga.verbose = True

NameError: name 'GigaChat' is not defined

In [ ]:
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
messages = [
    SystemMessage(
        content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
    )
]


# Ввод пользователя
user_input = input("User: ")
messages.append(HumanMessage(content=user_input))
res = giga(messages)
messages.append(res)
# Ответ модели
print("Bot: ", res.content)

User: Привет


Bot:  Здравствуйте! Я могу вам чем-то помочь?


In [ ]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field


class SendSmsResult(BaseModel):
    status: str = Field(description="Статус отправки сообщения")
    message: str = Field(description="Сообщение о результате отправки SMS")


few_shot_examples = [
    {
        "request": "Можешь ли ты отправить SMS-сообщение на номер 123456789 с содержимым 'Привет, как дела?'",
        "params": {"recipient": "123456789", "message": "Привет, как дела?"},
    }
]


@tool(few_shot_examples=few_shot_examples)
def send_sms(
    recipient: str = Field(description="Номер телефона получателя"),
    message: str = Field(description="Содержимое сообщения"),
) -> SendSmsResult:
    """Отправить SMS-сообщение"""
    print(f"! send_sms to {recipient}, text: {message}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SendSmsResult(status="OK", message="Сообщение отправлено!")

In [ ]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

tools = [send_sms]
agent = create_gigachat_functions_agent(giga, tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
)

In [ ]:
import time


def chat(agent_executor):
    chat_history = [
        SystemMessage(
            content="Ты полезный ассистент. Выполняй команды пользователя, уточняй недостающие данные."
        )
    ]

    while True:
        user_input = input("Клиент: ")
        if user_input == "":
            break
        print(f"User: {user_input}")
        result = agent_executor.invoke(
            {
                "chat_history": chat_history,
                "input": user_input,
            }
        )
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=result["output"]))
        print("\033[93m" + f"Bot: {result['output']}" + "\033[0m")
        time.sleep(0.2)


chat(agent_executor)

Клиент: Отправь поздравление с днем рождения на номер +79992223344


User: Отправь поздравление с днем рождения на номер +79992223344


! send_sms to +79992223344, text: С днём рождения! Желаю счастья и успеха во всём.
Bot: Поздравление успешно отправлено на номер +79992223344.


KeyboardInterrupt: Interrupted by user

Несколько функций

In [ ]:
from typing import List, Optional


class SearchMoviesResult(BaseModel):
    movies: List[str] = Field(
        description="Список названий фильмов, соответствующих заданным критериям поиска"
    )


few_shot_examples = [
    {"request": "Найди все фильмы жанра комедия", "params": {"genre": "комедия"}}
]


@tool(few_shot_examples=few_shot_examples)
def search_movies(
    genre: Optional[str] = Field(None, description="Жанр фильма"),
    year: Optional[int] = Field(None, description="Год выпуска фильма"),
    actor: Optional[str] = Field(None, description="Имя актера, снимавшегося в фильме"),
) -> SearchMoviesResult:
    """Поиск фильмов на основе заданных критериев"""
    print(f"! search_movies genre {genre}, year: {year}, actor: {actor}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SearchMoviesResult(movies=["Опенгеймер"])

In [ ]:
class TripDistanceResult(BaseModel):
    distance: int = Field(
        description="Расстояние между начальным и конечным местоположением в километрах"
    )


few_shot_examples = [
    {
        "request": "Насколько далеко от Москвы до Санкт-Петербурга?",
        "params": {"start_location": "Москва", "end_location": "Санкт-Петербург"},
    }
]


@tool(few_shot_examples=few_shot_examples)
def calculate_trip_distance(
    start_location: str = Field(description="Начальное местоположение"),
    end_location: str = Field(description="Конечное местоположение"),
) -> TripDistanceResult:
    """Рассчитать расстояние между двумя местоположениями."""
    print(
        f"! calculate_trip_distance start_location {start_location}, end_location: {end_location}"
    )
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return TripDistanceResult(distance=650)

In [ ]:
tools = [send_sms, search_movies, calculate_trip_distance]
agent = create_gigachat_functions_agent(giga, tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
)

chat(agent_executor)

Клиент: Отправь на номер 98765678 смс с поздравлением поздравление с днем рождения


User: Отправь на номер 98765678 смс с поздравлением поздравление с днем рождения


! send_sms to 98765678, text: Поздравление с днем рождения
Bot: Сообщение успешно отправлено на номер 98765678.
Клиент: Посчитай расстояние между москвой и питером


User: Посчитай расстояние между москвой и питером


! calculate_trip_distance start_location Москва, end_location: Санкт-Петербург
Bot: Расстояние между Москвой и Санкт-Петербургом составляет 650 километров.
Клиент: Посоветуй фильм ужасов


...

User: Посоветуй фильм ужасов 2023 года


! search_movies genre ужасы, year: 2023, actor: description='Имя актера, снимавшегося в фильме' extra={}
Bot: Я не могу посоветовать вам фильм ужасов 2023 года, так как в списке нет подходящих вариантов.
Клиент: Найди фильмы Нолана за 2023


User: Найди фильмы Нолана за 2023


! search_movies genre description='Жанр фильма' extra={}, year: 2023, actor: description='Имя актера, снимавшегося в фильме' extra={}
Bot: В 2023 году вышел только один фильм с участием режиссёра Кристофера Нолана — это драма «Оппенгеймер».


KeyboardInterrupt: Interrupted by user

Добавляем функцию поиска

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

In [ ]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

agent = create_gigachat_functions_agent(giga, tools)

# AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
agent_executor.invoke(
    {"input": "Найди текущий курс биткоина и напечатай только число"}
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'текущий курс биткоина'}`




Используйте наш бесплатный конвертер для расчета BTC - USD. Текущий курс обмена BTC на USD составляет $64,023.78. Бесплатный конвертер в реальном времени на основе данных CoinMarketCap. Курс Bitcoin к доллару США BTC/USD на графике онлайн. Курс Bitcoin сегодня: $63,224.57 за 1 BTC. Цена BTC в USD изменилась за последние 24 часа на +0.88% . Текущая рыночная капитализация Bitcoin — $1.25 трлн при суммарном ... Курс Биткоина, Bitcoin (BTC) — графики и цены. Создавайте оповещения и уведомления на цену биткоина онлайн в реальном времени. Что происходит с ценой биткоина, почему курс обновил свой исторический максимум. Что такое халвинг, как часто происходит это событие, сколько их уже было. Стоит ли покупать биткоин в 2024 году. Текущий курс биткоина (btc) к доллару на биржах ... Курс биткоина на 13.07.2024 г.: 57 738,08 $ 1 биткоинов в рублях - Изменение за последний месяц ...64023.78

> Finished chain.


'64023.78'

Добавляем второго агента

In [ ]:
import pyfiglet

pyfiglet.print_figlet("Hello!", font="epic")

          _______  _        _        _______  _ 
|\     /|(  ____ \( \      ( \      (  ___  )( )
| )   ( || (    \/| (      | (      | (   ) || |
| (___) || (__    | |      | |      | |   | || |
|  ___  ||  __)   | |      | |      | |   | || |
| (   ) || (      | |      | |      | |   | |(_)
| )   ( || (____/\| (____/\| (____/\| (___) | _ 
|/     \|(_______/(_______/(_______/(_______)(_)
                                                



In [ ]:
from langchain.tools import tool


@tool
def draw_banner(number: str) -> str:
    """Рисует баннер с текстом результатов кода в виде Ascii-графики

    Args:
        number (str): Число, которое нужно нарисовать на баннере
    """
    pyfiglet.print_figlet(number, font="epic")
    return "Draw complete"


new_tools = [search_tool, draw_banner]

In [ ]:
giga_pro = GigaChat(credentials="",
                model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

In [ ]:
agent = create_gigachat_functions_agent(giga_pro, new_tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=True,
)

agent_executor.invoke(
    {
        "input": "Найди в интернете курс биткоина в долларах и нарисуй это число на банере."
    }
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'курс биткоина в долларах'}`


Используйте наш бесплатный конвертер для расчета BTC - USD. Текущий курс обмена BTC на USD составляет $64,023.78. Бесплатный конвертер в реальном времени на основе данных CoinMarketCap. За год стоимость 100 биткоин в долларах увеличилась примерно на 3 698 993 доллара. Курс. 100 btc в USD. июль 2024. 66 224,43. 6 622 443,0441105. июнь 2024. 61 953,465. Узнайте сколько будет ₿1 Биткоин (btc) в Долларах США (usd) - на данный момент. Получите реальные курсы обмена, анализ и динамику изменений валютной пары на графике и в таблице. Получаете. Удобный конвертер криптовалют, вы можете узнать сколько стоит Bitcoin (BTC) в Доллар США (USD) и посчитать на онлайн калькуляторе нужные вам суммы. Bitcoin Історія Ціни. Наживо Bitcoin ціна сьогодні UAH2,717,731.06 UAH із 24-годинним обсягом торгів UAH1,476,242,288,721.18 UAH. Ми оновлюємо ціну BTC до UAH в режимі реального часу.
Invokin

'- вот банер с курсом биткоина в долларах.'